In [80]:
# 多元线性回归

## 安装并导入可能需要的模块

In [81]:
# !pip install cython
# !pip install scipy
# !pip install statsmodels

In [82]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 读取数据并查看数据特征

In [83]:
data = pd.read_csv('/home/aistudio/work/50_Startups.csv') # 读取数据
print(data.head())      # 查看前5条数据
print(data.columns)     # 查看列索引
print(data.index)       # 查看行索引
print(data.shape)       # 查看数据集大小 shape[0]表示行数 shape[1]表示列数
print(data.describe())  # 查看数据集总体的一些统计特征 如数量、均值、标准差、分位点

   R&D Spend  Administration  Marketing Spend       State     Profit
0  165349.20       136897.80        471784.10    New York  192261.83
1  162597.70       151377.59        443898.53  California  191792.06
2  153441.51       101145.55        407934.54     Florida  191050.39
3  144372.41       118671.85        383199.62    New York  182901.99
4  142107.34        91391.77        366168.42     Florida  166187.94
Index(['R&D Spend', 'Administration', 'Marketing Spend', 'State', 'Profit'], dtype='object')
RangeIndex(start=0, stop=50, step=1)
(50, 5)
           R&D Spend  Administration  Marketing Spend         Profit
count      50.000000       50.000000        50.000000      50.000000
mean    73721.615600   121344.639600    211025.097800  112012.639200
std     45902.256482    28017.802755    122290.310726   40306.180338
min         0.000000    51283.140000         0.000000   14681.400000
25%     39936.370000   103730.875000    129300.132500   90138.902500
50%     73051.080000   122699.7950

## 选取自变量、因变量

In [84]:
# 选择自变量X
# ['R&D Spend', 'Administration', 'Marketing Spend', 'State']
# ['研发支出',   '管理费用',        '市场营销',        '州']
# 选择因变量y
X = dataset.iloc[:, :-1].values # 自变量
y = dataset.iloc[:, 4].values # profit 利润

## 将数据中分类数据进行转换

In [85]:
# 由常识知道State是分类数据,下面查看State列的可能取值有几种
dataset[["State"]].nunique()

State    3
dtype: int64

In [86]:
# 下面进行分类数据 分别采用OneHotEncoder、LabelEncoder进行编码
# 首先导入编码需要的模块
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

- 函数原型
```
class sklearn.compose.ColumnTransformer(transformers, remainder=’drop’, sparse_threshold=0.3, n_jobs=None, transformer_weights=None)
```
- 参数transformers:该参数是一个由元组组成的列表(list of tuples)，每个元组的结构为：(name, transformer, column):
	- name: transformer的名字，随便起一个字符串即可；
	- transformer: 支持 fit 和 transform 的estimator或者 passthrough 或者 drop. 
	- column: 指定对哪些列做转换操作，所以可以是下标、列名等。
- 参数remainder:这个参数的值可以是支持 fit 和 transform 的estimator或者 passthrough 或者 drop ，默认值是drop，其功能和transformers参数非常像：
  - drop：表示将column指定的列之外的其他列都丢弃；
  - passthrough：表示将column指定的列之外的其他列透传；
  - estimator：表示对column指定的列之外的其他列执行该estimator代表的转换。


In [87]:
# 采用独热编码转换 两种实现方案
# 实现独热编码的方法一
# 先做一个编码器
from sklearn.compose import ColumnTransformer
st=ColumnTransformer([('State',OneHotEncoder(),[3])],remainder='passthrough')
# 在使用这个编码器对X编码 编码后的数据
X = st.fit_transform(X)
# X['State']可能取值集合：{'New York','California','Florida'} 共三种可能情况 因此独热码有3位标识
# 0.0.1表示New York 1.0.0表示California 0.1.0表示Florida

In [88]:
# 实现独热编码的方法二
enc = OneHotEncoder()
enc.fit(X[:,3:4])                           # fit来学习编码
code = enc.transform(X[:,3:4]).toarray()    # 进行编码
X = np.append(X, code, axis=1)              # 编码追加到X
X = np.delete(X, 3, axis=1)                 # 删除原来的列
X[0:3]

array([[0.0, 0.0, 1.0, 136897.8, 471784.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 1.0],
       [1.0, 0.0, 0.0, 151377.59, 443898.53, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 1.0, 0.0],
       [0.0, 1.0, 0.0, 101145.55, 407934.54, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 1.0, 0.0, 0.0]], dtype=object)

In [89]:
# 另一种编码方式 LabelEncodeer
labelencoder_X = LabelEncoder()
X[:,3] = labelencoder_X.fit_transform(X[:,3])

## 划分训练集、测试集

- 训练集相当于上课学知识
- 验证集相当于课后的的练习题，用来纠正和强化学到的知识
- 测试集相当于期末考试，用来最终评估学习效果

In [90]:
# from sklearn.cross_validation import train_test
# 注意：上面的这种导入方式在目前版本的sklearn种已经被弃用
# 可以使用的导入方式应该是下面这样的
# from sklearn.model_selection import train_test_split

In [91]:
# 导入划分训练集与测试集所需要的模块
from sklearn.model_selection import train_test_split
# 划分训练集与测试集 比例是train:test=8:2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [92]:
print("X_train：",X_train[0:5]) # 查看训练集特征前5条
print("y_train：",y_train[0:5]) # 查看训练集标签前5条
print("X_test：",X_test[0:5])  # 查看测试集特征前5条
print("y_test：",y_test[0:5])  # 查看测试集标签前5条

X_train： [[0.0 1.0 0.0 12 214634.81 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0]
 [0.0 0.0 1.0 5 205517.64 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0]
 [0.0 1.0 0.0 36 134050.07 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0]
 [1.0 0.0 0.0 48 210797.67 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
  0.0 0.0]
 [0.0 1.0 0.0 17 294919.57 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 

## 多元线性回归模型

In [93]:
from sklearn.linear_model import LinearRegression # 线性回归模型
model = LinearRegression()     # 实例化一个模型
model.fit(X_train, y_train)    # 训练拟合模型

/opt/conda/envs/python35-paddle120-env/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [94]:
y_test_pred = model.predict(X_test)    # 在测试集上使用模型进行预测
y_train_pred = model.predict(X_train)  # 在训练集上使用模型进行预测
print(y_test_pred[0:5])    # 输出测试集前5条预测结果
print(y_train_pred[0:5])   # 输出训练集前5条预测结果

[161471.64664003  61943.24973421  80223.48879458  40547.59593576
  92719.75013245]
[ 96778.91999999  96479.50999999 105733.53999996  96712.79999999
 124266.90000003]


### **以上为all in的建模方法**
![](https://ai-studio-static-online.cdn.bcebos.com/6af0045a9db14778968eadc3d26564533efbbddb164442e6aa6e17c08f59ef35)

### **下面是Backward  Elimination建模方法**
![](https://ai-studio-static-online.cdn.bcebos.com/478d1c5fd30f409692120aafaa9716942f21828eb1364de384f8973fa3e0fdf8)


In [95]:
#导入标准库
import statsmodels.formula.api as sm

In [96]:
#由于标准库中的函数是不包含常数项的，所以需要手动加上一列
X_train = np.append(arr = np.ones((40, 1)).astype(int), values = X_train, axis = 1)
# 在[1,1,...,1]T追加X_train 并把结果保存在X_train中

**下面代码有误，请修改，使之正确**

**请写出修订后代码及说明**

In [97]:
X_train = pd.DataFrame(X_train).astype(float)   # 转DataFrame
X_opt = X_train.iloc[:, [0, 1, 2, 3, 4, 5]]     # 变量替换

In [98]:
# 第一种方法
import statsmodels.api as sm      # 导入模块
model = sm.OLS(y_train, X_opt)    # 最小二乘建模
regressor_OLS = model.fit()       # 拟合模型
regressor_OLS.summary()           # 评价模型

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                     17.13
Date:                Sun, 11 Jul 2021   Prob (F-statistic):           7.19e-08
Time:                        18:41:19   Log-Likelihood:                -459.39
No. Observations:                  40   AIC:                             928.8
Df Residuals:                      35   BIC:                             937.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
0           2.842e+04   8112.952      3.503      0.001    1.19e+04    4.49e+04
1           8070.3776   5731.650      1.408      0.168   -3565.491    1.97e+04
2           2882.3317   7149.728      0.403      0.689   -1.16e+04    1.74e+04
3           1.747e+04   6205.202      2.815      0.008    4869.251    3.01e+04
4            810.2389    289.491      2.799      0.008     222.540    1397.938
5              0.2475      0.033      7.547      0.000       0.181       0.314
==============================================================================
Omnibus:                        5.355   Durbin-Watson:                   1.915
Prob(Omnibus):                  0.069   Jarque-Bera (JB):                4.410
Skew:                          -0.501   Prob(JB):                        0.110
Kurtosis:                       4.281   Cond. No.                     9.56e+17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.46e-24. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [99]:
# 第二种方法
import statsmodels.formula.api as sm     # 导入模块
model = sm.ols('y_train~X_opt', X_opt)   # 最小二乘建模
regressor_OLS = model.fit()              # 拟合模型
regressor_OLS.summary()                  # 评价模型

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                y_train   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                     17.13
Date:                Sun, 11 Jul 2021   Prob (F-statistic):           7.19e-08
Time:                        18:41:19   Log-Likelihood:                -459.39
No. Observations:                  40   AIC:                             928.8
Df Residuals:                      35   BIC:                             937.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.624e+04   4635.972      3.503      0.001    6828.013    2.57e+04
X_opt[0]    1.624e+04   4635.972      3.503      0.001    6828.013    2.57e+04
X_opt[1]    4010.4933   5481.293      0.732      0.469   -7117.123    1.51e+04
X_opt[2]   -1177.5526   6627.902     -0.178      0.860   -1.46e+04    1.23e+04
X_opt[3]    1.341e+04   5807.357      2.309      0.027    1617.036    2.52e+04
X_opt[4]     810.2389    289.491      2.799      0.008     222.540    1397.938
X_opt[5]       0.2475      0.033      7.547      0.000       0.181       0.314
==============================================================================
Omnibus:                        5.355   Durbin-Watson:                   1.915
Prob(Omnibus):                  0.069   Jarque-Bera (JB):                4.410
Skew:                          -0.501   Prob(JB):                        0.110
Kurtosis:                       4.281   Cond. No.                     3.66e+22
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.68e-33. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [100]:
X_opt = X_train.iloc[:, [0, 1, 3, 4, 5]]
model = sm.ols('y_train~X_opt', X_opt)   # 最小二乘建模
regressor_OLS = model.fit()              # 拟合模型
regressor_OLS.summary()                  # 评价模型

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                y_train   R-squared:                       0.662
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                     17.13
Date:                Sun, 11 Jul 2021   Prob (F-statistic):           7.19e-08
Time:                        18:41:19   Log-Likelihood:                -459.39
No. Observations:                  40   AIC:                             928.8
Df Residuals:                      35   BIC:                             937.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.565e+04   6645.829      2.355      0.024    2159.011    2.91e+04
X_opt[0]    1.565e+04   6645.829      2.355      0.024    2159.011    2.91e+04
X_opt[1]    5188.0458   1.03e+04      0.503      0.618   -1.58e+04    2.61e+04
X_opt[2]    1.458e+04   1.05e+04      1.393      0.172   -6671.773    3.58e+04
X_opt[3]     810.2389    289.491      2.799      0.008     222.540    1397.938
X_opt[4]       0.2475      0.033      7.547      0.000       0.181       0.314
==============================================================================
Omnibus:                        5.355   Durbin-Watson:                   1.915
Prob(Omnibus):                  0.069   Jarque-Bera (JB):                4.410
Skew:                          -0.501   Prob(JB):                        0.110
Kurtosis:                       4.281   Cond. No.                     1.88e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 6.32e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [101]:
X_opt = X_train.iloc[:, [0, 3, 4, 5]]
model = sm.ols('y_train~X_opt', X_opt)   # 最小二乘建模
regressor_OLS = model.fit()              # 拟合模型
regressor_OLS.summary()                  # 评价模型

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                y_train   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.631
Method:                 Least Squares   F-statistic:                     23.24
Date:                Sun, 11 Jul 2021   Prob (F-statistic):           1.52e-08
Time:                        18:41:19   Log-Likelihood:                -459.54
No. Observations:                  40   AIC:                             927.1
Df Residuals:                      36   BIC:                             933.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1.754e+04   5419.048      3.237      0.003    6553.635    2.85e+04
X_opt[0]    1.754e+04   5419.048      3.237      0.003    6553.635    2.85e+04
X_opt[1]     1.14e+04   8254.505      1.381      0.176   -5338.531    2.81e+04
X_opt[2]     811.0712    286.467      2.831      0.008     230.090    1392.052
X_opt[3]       0.2444      0.032      7.666      0.000       0.180       0.309
==============================================================================
Omnibus:                        6.335   Durbin-Watson:                   1.919
Prob(Omnibus):                  0.042   Jarque-Bera (JB):                5.908
Skew:                          -0.527   Prob(JB):                       0.0521
Kurtosis:                       4.560   Cond. No.                     1.94e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.99e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [102]:
X_opt = X_train.iloc[:, [0, 3, 5]]
model = sm.ols('y_train~X_opt', X_opt)   # 最小二乘建模
regressor_OLS = model.fit()              # 拟合模型
regressor_OLS.summary()                  # 评价模型

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                y_train   R-squared:                       0.584
Model:                            OLS   Adj. R-squared:                  0.561
Method:                 Least Squares   F-statistic:                     25.94
Date:                Sun, 11 Jul 2021   Prob (F-statistic):           9.10e-08
Time:                        18:41:19   Log-Likelihood:                -463.56
No. Observations:                  40   AIC:                             933.1
Df Residuals:                      37   BIC:                             938.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2.782e+04   4390.347      6.336      0.000    1.89e+04    3.67e+04
X_opt[0]    2.782e+04   4390.347      6.336      0.000    1.89e+04    3.67e+04
X_opt[1]    1.203e+04   8999.931      1.337      0.189   -6204.146    3.03e+04
X_opt[2]       0.2454      0.035      7.057      0.000       0.175       0.316
==============================================================================
Omnibus:                        4.331   Durbin-Watson:                   2.017
Prob(Omnibus):                  0.115   Jarque-Bera (JB):                4.136
Skew:                          -0.186   Prob(JB):                        0.126
Kurtosis:                       4.531   Cond. No.                     4.26e+18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.24e-25. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [103]:
X_opt = X_train.iloc[:, [0, 3]]
model = sm.ols('y_train~X_opt', X_opt)   # 最小二乘建模
regressor_OLS = model.fit()              # 拟合模型
regressor_OLS.summary()                  # 评价模型

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                y_train   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.9072
Date:                Sun, 11 Jul 2021   Prob (F-statistic):              0.347
Time:                        18:41:19   Log-Likelihood:                -480.61
No. Observations:                  40   AIC:                             965.2
Df Residuals:                      38   BIC:                             968.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   5.246e+04   4023.326     13.038      0.000    4.43e+04    6.06e+04
X_opt[0]    5.246e+04   4023.326     13.038      0.000    4.43e+04    6.06e+04
X_opt[1]    1.295e+04   1.36e+04      0.952      0.347   -1.46e+04    4.05e+04
==============================================================================
Omnibus:                        0.065   Durbin-Watson:                   1.954
Prob(Omnibus):                  0.968   Jarque-Bera (JB):                0.117
Skew:                          -0.078   Prob(JB):                        0.943
Kurtosis:                       2.786   Cond. No.                     4.46e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 4.3e-30. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

## 总结问题原因，并加以说明

查阅[资料](https://www.statsmodels.org/stable/api.html)发现statsmodels提供了两种ols：

一种是在`statsmodels.api`中的[`OLS方法`](https://www.statsmodels.org/stable/generated/statsmodels.regression.linear_model.OLS.html#statsmodels.regression.linear_model.OLS),

一种是在`statsmodels.formula.api`中的[`ols方法`](https://www.statsmodels.org/stable/generated/statsmodels.formula.api.ols.html#statsmodels.formula.api.ols)

**上面的程序出错是因为ols方法接受的数据类型不匹配，我查看来一下发现本来X_opt 是ndarray，每个元素是Object类型
我把它改成DataFrame类型并且把每个数字改为float就行了**
